# Remez approximation 

Remez algorithm [1] is an algorithm for approximating a smooth function with a polynomial of given degree on a given interval that minimizes $L_{\infty}$ error.

The paper [2] suggests how to extend it to get piecewise polynomial approximation with bounded  $L_{\infty}$ error that minimizes number of pieces.

Here (in `remez.py`) I implemented this algorithm.


## Experiment

In this notebook, I verify that this algorithm gives correct approximation within given tolerance for some functions and reproduce results from table II in [2].

Experiment was run for:
  * Functions: $\tanh(x), \exp(-x^2), \sin(x), \exp(-x), \arcsin(x)$.
  * Degrees: $3,4,5,6$.
  * Tolerances: $10^{-5}, 10^{-7}, 10^{-9}. $
  * Interval [-0.5, 0.5].

For each combination of parameters I built piecewise approximation, verified that $L_{\infty}$ error is lower than requested error threshold, and printed the number of intervals (pieces) in the piecewise approximation.

## Odd/even trick

To correctly reproduce results I had to implement the trick allowing to compute only even/odd coefficients for even/odd functions.

If $f(x)$ is even ($f(x)=f(-x)$), then we consider $g(x)=f(\sqrt{x})$, get approximation for it and then evaluate $f(x) = g(x^2)$.

If $f(x)$ is even ($f(x)=-f(-x)$), then we consider $g(x)=f(\sqrt{x})/\sqrt{x}$, get approximation for it and then evaluate $f(x) = x \cdot g(x^2)$.

## Results

I got significantly better results (lower number of subintervals) than reported in that paper. The only exeception is for arcsine, for degree 3 and tolerance $10^{-9}$ where I got 8 intervals, while the paper reports 6.


### Referecnes

[1] Eugene Y Remez. Sur la determination des polynomes dapproximation de degre donnee. Comm. Soc. Math. Kharkov, 10:41–63, 1934.

[2] Thomas Haner, Martin Roetteler, Krysta M. Svore. Optimizing Quantum Circuits for Arithmetic. 2018. https://arxiv.org/abs/1805.12445


In [1]:
import numpy as np 
import math

from remez import remez_piecewise
from remez_test import _linf_error


FUNCTIONS = [
   ("tanh(x)", lambda x: np.tanh(x), "odd"),
   ("exp(-x^2)", lambda x: np.exp(-x**2), "even"),
   ("sin(x)", lambda x: np.sin(x), "odd"),
   ("exp(-x)", lambda x: np.exp(-x), ""),
   ("arcsin(x)", lambda x: np.arcsin(x), "odd"), 
]

def approx_even_function(fx, interval, degree, tol):
    assert interval[0] == -interval[1]
    pieces = remez_piecewise(lambda t: fx(t**0.5), (0, interval[1]**2), degree, tol/2)
    return pieces, lambda x: pieces.eval(x**2)

def approx_odd_function(fx, interval, degree, tol):
    assert interval[0] == -interval[1]
    pieces = remez_piecewise(lambda t: fx(np.sqrt(t)) / np.sqrt(t), (1e-30, interval[1]**2), degree, tol/2)
    return pieces, lambda x: x*pieces.eval(x**2)

interval = (-0.5, 0.5)
tolerances =  [1e-5, 1e-7, 1e-9]
degrees =  [3, 4, 5, 6]

table_header = ["Function", "L∞ error", "Polynomial degree", "# of subintervals", "# of subintervals with odd/even trick"]
table = []

for function_name, fx, parity in FUNCTIONS:
    for tol in tolerances:
        for degree in degrees:
            pieces = remez_piecewise(fx, interval, degree, tol)
            err = _linf_error(fx, lambda x: pieces.eval(x), interval)
            assert err <=  tol
            row = [function_name, str(tol), degree, len(pieces.pieces)]

            if parity == "even":
                pieces, f_approx = approx_even_function(fx, interval, degree, tol)
                assert _linf_error(fx, f_approx, interval) <= tol
                row += [len(pieces.pieces)]
            elif parity == "odd":
                pieces, f_approx = approx_odd_function(fx, interval, degree, tol)
                err = _linf_error(fx, f_approx, interval) 
                #print(err, tol)
                assert err <= tol
                row += [len(pieces.pieces)]
            else:
                row += [""]

            table.append(row)

import pandas as pd
df = pd.DataFrame(table, columns=table_header)
df = df.style.hide(axis="index")
df

Function,L∞ error,Polynomial degree,# of subintervals,# of subintervals with odd/even trick
tanh(x),1e-05,3,3,1
tanh(x),1e-05,4,2,1
tanh(x),1e-05,5,1,1
tanh(x),1e-05,6,1,1
tanh(x),1e-07,3,10,2
tanh(x),1e-07,4,5,1
tanh(x),1e-07,5,3,1
tanh(x),1e-07,6,2,1
tanh(x),1e-09,3,31,6
tanh(x),1e-09,4,10,2
